In [27]:
import pandas as pd

In [28]:
# determined encodings with `file --exclude encoding Billboard.csv`
df = pd.read_csv("./data/Billboard.csv", low_memory=False, encoding="ISO-8859-1")
df.head()

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,2015,262,h,2015-001,25.0,1.0,0.0,1.0,39,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,298,h,2015-002,34.0,27.0,2.0,2.0,10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,301,h,2015-003,32.0,25.0,10.0,1.0,7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,304,h,2015-004,16.0,5.0,0.0,1.0,18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,308,h,2015-005,20.0,0.0,0.0,1.0,44,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
cols_keep = df.columns[:125]

In [31]:
df = df[cols_keep]
df.head()

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,...,82nd Week,83rd Week,84th Week,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week
0,2015,262,h,2015-001,25.0,1.0,0.0,1.0,39,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,298,h,2015-002,34.0,27.0,2.0,2.0,10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,301,h,2015-003,32.0,25.0,10.0,1.0,7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,304,h,2015-004,16.0,5.0,0.0,1.0,18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,308,h,2015-005,20.0,0.0,0.0,1.0,44,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_basics = df.iloc[:, :34]
df_basics.head()

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,...,Label/Number,Media,Stereo (55-68),Pic Sleeve,Genre,Comments,Written By,Temp 1,Date Entered,Date Peaked
0,2015,262,h,2015-001,25.0,1.0,0.0,1.0,39,NaN,...,Lichtdicht Digital,DD,NaN,NaN,NaN,NaN,C.Rehbein,1079,8/9/14,1/3/15
1,2015,298,h,2015-002,34.0,27.0,2.0,2.0,10,NaN,...,Decaydance/Island Digital,DD,NaN,NaN,NaN,NaN,"J.R.Rotem,P.V.Stump,P.Wentz,J.Trohman,A.Hurley...",2467,9/27/14,2/7/15
2,2015,301,h,2015-003,32.0,25.0,10.0,1.0,7,NaN,...,Safehouse/Island Digital,DD,NaN,NaN,NaN,NaN,"N.Jonas,N.Lambrozza,S.Wilcox",2383,9/27/14,1/24/15
3,2015,304,h,2015-004,16.0,5.0,0.0,1.0,18,NaN,...,Epic Digital,DD,NaN,NaN,NaN,NaN,"M.Trainor,K.Kadish",583,9/27/14,5/16/15
4,2015,308,h,2015-005,20.0,0.0,0.0,1.0,44,NaN,...,Republic Nashville Digital,DD,NaN,NaN,NaN,NaN,"C.R.Barlowe,J.Frasure,S.Buxton,T.Hubbard,B.Kelley",770,10/4/14,2/7/15


In [61]:
len(df_basics)

39982

In [60]:
df_basics["Prefix"].nunique()

39982

In [68]:
# list of writers
df_basics['Written By'].str.split(",", 10, expand=True)

,0,1,2,3,4,5,6,7,8,9,10
0,C.Rehbein,None,None,None,None,None,None,None,None,None,None
1,J.R.Rotem,P.V.Stump,P.Wentz,J.Trohman,A.Hurley,M.J.Fonseca,R.Kumari,J.Trantner,S.Vega,None,None
2,N.Jonas,N.Lambrozza,S.Wilcox,None,None,None,None,None,None,None,None
3,M.Trainor,K.Kadish,None,None,None,None,None,None,None,None,None
4,C.R.Barlowe,J.Frasure,S.Buxton,T.Hubbard,B.Kelley,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
39977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
selected_columns = [0, 10, 16, 24, 28, 30] + list(range(34, 125))

In [45]:
df_weekly = df.iloc[:,selected_columns]
df_weekly.head()

,Year,Artist,Track,Label/Number,Genre,Written By,1st Week,2nd Week,3rd Week,4th Week,...,82nd Week,83rd Week,84th Week,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week
0,2015,Milky Chance,Stolen Dance,Lichtdicht Digital,NaN,C.Rehbein,95,91,84,71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,Fall Out Boy,Centuries,Decaydance/Island Digital,NaN,"J.R.Rotem,P.V.Stump,P.Wentz,J.Trohman,A.Hurley...",22,53,55,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,Nick Jonas,Jealous,Safehouse/Island Digital,NaN,"N.Jonas,N.Lambrozza,S.Wilcox",78,80,58,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,Meghan Trainor,Dear Future Husband,Epic Digital,NaN,"M.Trainor,K.Kadish",94,95,93,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,Florida Georgia Line,Sun Daze,Republic Nashville Digital,NaN,"C.R.Barlowe,J.Frasure,S.Buxton,T.Hubbard,B.Kelley",83,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df_weekly_T = df_weekly.transpose()

In [50]:
headers = df_weekly_T.iloc[2,:]
headers

0                               Stolen Dance
1                                  Centuries
2                                    Jealous
3                        Dear Future Husband
4                                   Sun Daze
                        ...                 
39977                   Pat Kelly on a Spree
39978    Pat Kelly's Plea in His Own Defense
39979                         Semper Fidelis
39980                        Washington Post
39981                          The Thunderer
Name: Track, Length: 39982, dtype: object

In [53]:
df_weekly_T.columns = headers
df_weekly_T.head(20)

Track,Stolen Dance,Centuries,Jealous,Dear Future Husband,Sun Daze,Heroes (We Could Be),Shotgun Rider,Thinking Out Loud,Body Language,Often,...,Pat Kelly as a Police Justice,Little 'Liza Loves You,"Home, Sweet Home",The Picture Turned to the Wall,Uncle Jefferson,Pat Kelly on a Spree,Pat Kelly's Plea in His Own Defense,Semper Fidelis,Washington Post,The Thunderer
Year,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,...,1891,1891,1891,1891,1891,1891,1891,1890,1890,1890
Artist,Milky Chance,Fall Out Boy,Nick Jonas,Meghan Trainor,Florida Georgia Line,Alesso,Tim McGraw,Ed Sheeran,Kid Ink,"Weeknd, The",...,Dan Kelly,Len Spencer,John Yorke Atlee,George J. Gaskin,Billy Golden,Dan Kelly,Dan Kelly,U.S. Marine Band,U.S. Marine Band,U.S. Marine Band
Track,Stolen Dance,Centuries,Jealous,Dear Future Husband,Sun Daze,Heroes (We Could Be),Shotgun Rider,Thinking Out Loud,Body Language,Often,...,Pat Kelly as a Police Justice,Little 'Liza Loves You,"Home, Sweet Home",The Picture Turned to the Wall,Uncle Jefferson,Pat Kelly on a Spree,Pat Kelly's Plea in His Own Defense,Semper Fidelis,Washington Post,The Thunderer
Label/Number,Lichtdicht Digital,Decaydance/Island Digital,Safehouse/Island Digital,Epic Digital,Republic Nashville Digital,Refune Digital,McGraw Digital (Big Machine),Atlantic Digital,Tha Alumni Group/88 Classic Digital,XO Digital (Republic),...,Ohio (No Number),Columbia (No Number),Columbia (No Number),North American (No Number),Columbia (No Number),Ohio (No Number),Ohio (No Number),Columbia (No Number),Columbia (No Number),Columbia (No Number)
Genre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Written By,C.Rehbein,"J.R.Rotem,P.V.Stump,P.Wentz,J.Trohman,A.Hurley...","N.Jonas,N.Lambrozza,S.Wilcox","M.Trainor,K.Kadish","C.R.Barlowe,J.Frasure,S.Buxton,T.Hubbard,B.Kelley","A.Lindblad,T.Lo,D.Bowie,B.Eno","H.Lindsey,M.Green,T.Verges","E.Sheeran,A.Wadge","B.T.Collins,M.S.Eriksen,T.E.Hermansen,M.A.Hoib...","A.Tesfaye,B.Diehl,J.Quenneville,A.Balshe,D.Sch...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st Week,95,22,78,94,83,89,97,69,100,97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Week,91,53,80,95,NaN,80,84,79,100,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3rd Week,84,55,58,93,NaN,78,76,75,88,99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4th Week,71,54,40,NaN,NaN,72,65,84,87,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df_main = df_weekly_T.iloc[6:, :]

In [59]:
df_main.iloc[:,:50]

Track,Stolen Dance,Centuries,Jealous,Dear Future Husband,Sun Daze,Heroes (We Could Be),Shotgun Rider,Thinking Out Loud,Body Language,Often,...,My Baby's Got A Smile On Her Face,Feeling Myself,Velvet,Lost Without You,Masterpiece,Soldier,Lost Stars,Get On Your Knees,Apparently,Lonely Eyes
1st Week,95,22,78,94,83,89,97,69,100,97,...,34,43,53,63,65,74,83,88,90,95
2nd Week,91,53,80,95,NaN,80,84,79,100,95,...,NaN,59,NaN,NaN,68,NaN,NaN,NaN,94,77
3rd Week,84,55,58,93,NaN,78,76,75,88,99,...,NaN,59,NaN,NaN,NaN,NaN,NaN,NaN,86,73
4th Week,71,54,40,NaN,NaN,72,65,84,87,86,...,NaN,61,NaN,NaN,NaN,NaN,NaN,NaN,79,73
5th Week,66,42,28,NaN,NaN,73,66,68,80,86,...,NaN,61,NaN,NaN,NaN,NaN,NaN,NaN,81,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87th Week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88th Week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89th Week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90th Week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
